In [1]:
# %%bash
# source ../my_genv/bin/activate &&
# python -m ipykernel install --user --name=my_genv

In [1]:
%matplotlib inline 
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import time

### DEFINE INPUTS, INCLUDING SCENARIO NAMES

In [2]:
# file and data paths
respath = '../gridpath82/scenarios/'
figpath = 'figures/'
datapathout = 'diagnostic_data/'

# figure and data suffix
suffix = time.strftime('%m%d%y')

# scenarios
scs = os.listdir(respath)
scs = [i for i in scs if 'fix' not in i]

# reference scenario
ref = 'prm_gg_ref_l1'

save_outputs = 'no' # 'yes' 'no'

# define load_zones
mercosur = ['Argentina','Brazil','Chile','Paraguay','Uruguay']
load_zone_name = 'mercosur'

# planning periods
periods = [2020,2025,2030,2035,2040,2045,2050]

# Figure aesthetics

In [3]:
# Colors
coal_color = '#343a40'
gas_color = '#6c757d'
hydro_color = '#2a648a'
solar_color = '#ef9226'
wind_color = '#8dc0cd'
nuclear_color = 'darkmagenta'
diesel_color = 'darkred'
other_color = '#f07167'
biomass_color = '#6ba661'
geothermal_color = 'slateblue'
battery_color = '#e7c41f'
pstorage_color = '#6a96ac'
trans_color = 'tan'
curtailment_color = 'red'


# Existing caps

In [4]:
list_pt_all_cap_tech = []

for i in range(len(scs)):
    all_cap = pd.read_csv(respath+str(scs[i])+'/results/capacity_all.csv')
    
    ###rename tech of existing and new capacities for plotting 
    all_cap.loc[(all_cap['capacity_type'] == 'gen_new_lin') & (all_cap['technology'] == 'Coal_Sub'),'technology'] = 'Coal_new'
    all_cap.loc[(all_cap['capacity_type'] == 'gen_new_lin') & (all_cap['technology'] == 'Coal_Sup'),'technology'] = 'Coal_new'
    all_cap.loc[(all_cap['capacity_type'] == 'gen_new_lin') & (all_cap['technology'] == 'Coal_IGCC'),'technology'] = 'Coal_new'
    all_cap.loc[(all_cap['capacity_type'] == 'gen_new_lin') & (all_cap['technology'] == 'Gas_CCGT'),'technology'] = 'Gas_new'
    all_cap.loc[(all_cap['capacity_type'] == 'gen_new_lin') & (all_cap['technology'] == 'Gas_ICE'),'technology'] = 'Gas_new'
    all_cap.loc[(all_cap['capacity_type'] == 'gen_new_lin') & (all_cap['technology'] == 'Gas_OCGT'),'technology'] = 'Gas_new'
    all_cap.loc[(all_cap['capacity_type'] == 'gen_new_lin') & (all_cap['technology'] == 'SolarPV'),'technology'] = 'Solar_new'
    all_cap.loc[(all_cap['capacity_type'] == 'gen_new_lin') & (all_cap['technology'] == 'SolarCSP'),'technology'] = 'Solar_new'
    all_cap.loc[(all_cap['capacity_type'] == 'gen_new_lin') & (all_cap['technology'] == 'Wind'),'technology'] = 'Wind_new'
    all_cap.loc[(all_cap['capacity_type'] == 'gen_new_bin') & (all_cap['technology'] == 'Hydro'),'technology'] = 'Hydro_new'
    all_cap.loc[(all_cap['capacity_type'] == 'gen_new_lin') & (all_cap['technology'] == 'Nuclear'),'technology'] = 'Nuclear_new'
    all_cap.loc[(all_cap['capacity_type'] == 'gen_new_lin') & (all_cap['technology'] == 'Diesel'),'technology'] = 'Diesel_new'
    all_cap.loc[(all_cap['capacity_type'] == 'gen_new_lin') & (all_cap['technology'] == 'Geothermal'),'technology'] = 'Geothermal_new'
    all_cap.loc[(all_cap['capacity_type'] == 'gen_new_lin') & (all_cap['technology'] == 'Biomass'),'technology'] = 'Biomass_new'
    
    ### CCS techs: all CCS techs assumed to be "new", no existing capacity

    all_cap.loc[(all_cap['capacity_type'] == 'gen_spec') & (all_cap['technology'] == 'Coal_Sub'),'technology'] = 'Coal'
    all_cap.loc[(all_cap['capacity_type'] == 'gen_spec') & (all_cap['technology'] == 'Coal_Sup'),'technology'] = 'Coal'
    all_cap.loc[(all_cap['capacity_type'] == 'gen_spec') & (all_cap['technology'] == 'Coal_IGCC'),'technology'] = 'Coal'
    all_cap.loc[(all_cap['capacity_type'] == 'gen_spec') & (all_cap['technology'] == 'Gas_CCGT'),'technology'] = 'Gas'
    all_cap.loc[(all_cap['capacity_type'] == 'gen_spec') & (all_cap['technology'] == 'Gas_ICE'),'technology'] = 'Gas'
    all_cap.loc[(all_cap['capacity_type'] == 'gen_spec') & (all_cap['technology'] == 'Gas_OCGT'),'technology'] = 'Gas'
    all_cap.loc[(all_cap['capacity_type'] == 'gen_spec') & (all_cap['technology'] == 'SolarPV'),'technology'] = 'Solar'
    all_cap.loc[(all_cap['capacity_type'] == 'gen_spec') & (all_cap['technology'] == 'SolarCSP'),'technology'] = 'Solar'
    all_cap.loc[(all_cap['capacity_type'] == 'gen_spec') & (all_cap['technology'] == 'Wind'),'technology'] = 'Wind'
    all_cap.loc[(all_cap['capacity_type'] == 'gen_spec') & (all_cap['technology'] == 'Hydro'),'technology'] = 'Hydro'
    all_cap.loc[(all_cap['capacity_type'] == 'gen_spec') & (all_cap['technology'] == 'Nuclear'),'technology'] = 'Nuclear'
    all_cap.loc[(all_cap['capacity_type'] == 'gen_spec') & (all_cap['technology'] == 'Diesel'),'technology'] = 'Diesel'
    all_cap.loc[(all_cap['capacity_type'] == 'gen_spec') & (all_cap['technology'] == 'Geothermal'),'technology'] = 'Geothermal'
    all_cap.loc[(all_cap['capacity_type'] == 'gen_spec') & (all_cap['technology'] == 'Biomass'),'technology'] = 'Biomass'
    
    ###total capacity by period and renamed tech
    all_cap_tech = all_cap.groupby(['period','technology'])['capacity_mw'].sum()/1000 
    all_cap_tech = all_cap_tech.reset_index()
    
    ###Creat pivot table
    pt_all_cap_tech = all_cap_tech.pivot(index='period', columns='technology', values='capacity_mw')
    
    ###assign scenario name in a column
    pt_all_cap_tech['scs'] = scs[i]

    ###Append to the pivot table to list
    list_pt_all_cap_tech.append(pt_all_cap_tech)

df_pt_all_cap_tech_allscs = pd.concat([list_pt_all_cap_tech[i] for i in range(len(scs))])
df_pt_all_cap_tech_allscs


technology  Battery  Biomass  Biomass_CCS  Biomass_IGCC  Biomass_new     Coal  \
period                                                                          
2020            0.0  15.0290          0.0           0.0          0.0  14.6090   
2025            0.0  15.0290          0.0           0.0          0.0  14.6090   
2030            0.0  14.9911          0.0           0.0          0.0  13.3964   
2035            0.0  14.2657          0.0           0.0          0.0  13.1804   
2040            0.0  14.2657          0.0           0.0          0.0  12.4724   
2045            0.0  13.7462          0.0           0.0          0.0  10.4902   
2050            0.0  13.7462          0.0           0.0          0.0  10.4902   
2020            0.0  15.0290          0.0           0.0          0.0  14.6090   
2025            0.0  15.0290          0.0           0.0          0.0  14.6090   
2030            0.0  14.9911          0.0           0.0          0.0  13.3964   
2035            0.0  14.2657          0.0           0.0          0.0  13.1804   
2040            0.0  14.2657          0.0           0.0          0.0  12.4724   
2045            0.0  13.7462          0.0           0.0          0.0  10.4902   
2050            0.0  13.7462          0.0           0.0          0.0  10.4902   
2020            0.0  15.0290          0.0           0.0          0.0  14.6090   
2025            0.0  15.0290          0.0           0.0          0.0  14.6090   
2030            0.0  14.9911          0.0           0.0          0.0  13.3964   
2035            0.0  14.2657          0.0           0.0          0.0  13.1804   
2040            0.0  14.2657          0.0           0.0          0.0  12.4724   
2045            0.0  13.7462          0.0           0.0          0.0  10.4902   
2050            0.0  13.7462          0.0           0.0          0.0  10.4902   
2020            0.0  15.0290          0.0           0.0          0.0  14.6090   
2025            0.0  15.0290          0.0           0.0          0.0  14.6090   
2030            0.0  14.9911          0.0           0.0          0.0  13.3964   
2035            0.0  14.2657          0.0           0.0          0.0  13.1804   
2040            0.0  14.2657          0.0           0.0          0.0  12.4724   
2045            0.0  13.7462          0.0           0.0          0.0  10.4902   
2050            0.0  13.7462          0.0           0.0          0.0  10.4902   
2020            0.0  15.0290          0.0           0.0          0.0  14.6090   
2025            0.0  15.0290          0.0           0.0          0.0  14.6090   
2030            0.0  14.9911          0.0           0.0          0.0  13.3964   
2035            0.0  14.2657          0.0           0.0          0.0  13.1804   
2040            0.0  14.2657          0.0           0.0          0.0  12.4724   
2045            0.0  13.7462          0.0           0.0          0.0  10.4902   
2050            0.0  13.7462          0.0           0.0          0.0  10.4902   
2020            0.0  15.0290          0.0           0.0          0.0  14.6090   
2025            0.0  15.0290          0.0           0.0          0.0  14.6090   
2030            0.0  14.9911          0.0           0.0          0.0  13.3964   
2035            0.0  14.2657          0.0           0.0          0.0  13.1804   
2040            0.0  14.2657          0.0           0.0          0.0  12.4724   
2045            0.0  13.7462          0.0           0.0          0.0  10.4902   
2050            0.0  13.7462          0.0           0.0          0.0  10.4902   

technology  Coal_CCS  Coal_new  Diesel  Diesel_CCS  ...  Geothermal_new  \
period                                              ...                   
2020             0.0       0.0  7.4730         0.0  ...             0.0   
2025             0.0       0.0  7.4730         0.0  ...             0.0   
2030             0.0       0.0  7.4730         0.0  ...             0.0   
2035             0.0       0.0  7.4730         0.0  ...     

In [5]:
df_pt_all_cap_tech_allscs = df_pt_all_cap_tech_allscs[['Battery','Biomass','Coal','Diesel','Gas','Geothermal','Hydro',
       'Nuclear','Solar','Wind','Biomass_CCS','Biomass_IGCC','Biomass_new','Coal_CCS','Coal_new','Diesel_CCS','Diesel_new',
       'Gas_CCS','Gas_new','Geothermal_new','Hydro_new','Nuclear_new','Solar_new', 'Wind_new','scs']]
# Shortening Scenario Names (if needed)
for i in range(len(df_pt_all_cap_tech_allscs)):
    df_pt_all_cap_tech_allscs.iloc[i,-1] = df_pt_all_cap_tech_allscs.iloc[i,-1][4:]
df_pt_all_cap_tech_allscs.iloc[:,-1]
ref_scs_tick = ref[4:]

In [6]:
# Flip this order to flip the year vs scenario grouping
x = [list(df_pt_all_cap_tech_allscs.index),
     list(df_pt_all_cap_tech_allscs.scs)]

# Existing gen capacity
fig = go.Figure(go.Bar(x=x, y=list(df_pt_all_cap_tech_allscs.Coal), name='Coal existing', marker_color=coal_color, marker_pattern_shape="x"))
fig.add_trace(go.Bar(x=x, y=list(df_pt_all_cap_tech_allscs.Gas), name='Gas existing', marker_color=gas_color, marker_pattern_shape="x"))
fig.add_trace(go.Bar(x=x, y=list(df_pt_all_cap_tech_allscs.Hydro), name='Hydro existing', marker_color=hydro_color, marker_pattern_shape="x"))
fig.add_trace(go.Bar(x=x, y=list(df_pt_all_cap_tech_allscs.Solar), name='Solar existing', marker_color=solar_color, marker_pattern_shape="x"))
fig.add_trace(go.Bar(x=x, y=list(df_pt_all_cap_tech_allscs.Wind), name='Wind existing', marker_color=wind_color, marker_pattern_shape="x"))
fig.add_trace(go.Bar(x=x, y=list(df_pt_all_cap_tech_allscs.Nuclear), name='Nuclear existing', marker_color=nuclear_color, marker_pattern_shape="x"))
fig.add_trace(go.Bar(x=x, y=list(df_pt_all_cap_tech_allscs.Biomass), name='Biomass existing', marker_color=biomass_color, marker_pattern_shape="x"))
fig.add_trace(go.Bar(x=x, y=list(df_pt_all_cap_tech_allscs.Diesel), name='Diesel existing', marker_color=diesel_color, marker_pattern_shape="x"))
fig.add_trace(go.Bar(x=x, y=list(df_pt_all_cap_tech_allscs.Geothermal), name='Geothermal existing', marker_color=geothermal_color, marker_pattern_shape="x"))
# fig.add_trace(go.Bar(x=x, y=list(df_pt_all_cap_tech_allscs.Battery), name='Battery existing', marker_color='black', marker_pattern_shape="/"))
# fig.add_trace(go.Bar(x=x, y=list(df_pt_all_cap_tech_allscs.PStorage), name='Pumped storage<br>existing', marker_color=pstorage_color, marker_pattern_shape="x"))

# New gen capacity
fig.add_trace(go.Bar(x=x, y=list(df_pt_all_cap_tech_allscs.Coal_new), name='Coal new', marker_color=coal_color))
fig.add_trace(go.Bar(x=x, y=list(df_pt_all_cap_tech_allscs.Coal_CCS), name='Coal CCS new', marker_color=coal_color, marker_pattern_shape="."))
fig.add_trace(go.Bar(x=x, y=list(df_pt_all_cap_tech_allscs.Gas_new), name='Gas new', marker_color=gas_color))
fig.add_trace(go.Bar(x=x, y=list(df_pt_all_cap_tech_allscs.Gas_CCS), name='Gas CCS new', marker_color=gas_color, marker_pattern_shape="."))
fig.add_trace(go.Bar(x=x, y=list(df_pt_all_cap_tech_allscs.Hydro_new), name='Hydro new', marker_color=hydro_color))
fig.add_trace(go.Bar(x=x, y=list(df_pt_all_cap_tech_allscs.Solar_new), name='Solar new', marker_color=solar_color))
fig.add_trace(go.Bar(x=x, y=list(df_pt_all_cap_tech_allscs.Wind_new), name='Wind new', marker_color=wind_color))
fig.add_trace(go.Bar(x=x, y=list(df_pt_all_cap_tech_allscs.Nuclear_new), name='Nuclear new', marker_color=nuclear_color))
fig.add_trace(go.Bar(x=x, y=list(df_pt_all_cap_tech_allscs.Biomass_new), name='Biomass new', marker_color=biomass_color))
fig.add_trace(go.Bar(x=x, y=list(df_pt_all_cap_tech_allscs.Biomass_CCS), name='Biomass CCS new', marker_color=biomass_color, marker_pattern_shape="."))
fig.add_trace(go.Bar(x=x, y=list(df_pt_all_cap_tech_allscs.Diesel_new), name='Diesel new', marker_color=diesel_color))
fig.add_trace(go.Bar(x=x, y=list(df_pt_all_cap_tech_allscs.Diesel_CCS), name='Diesel CCS new', marker_color=diesel_color, marker_pattern_shape="."))
fig.add_trace(go.Bar(x=x, y=list(df_pt_all_cap_tech_allscs.Geothermal_new), name='Geothermal new', marker_color=geothermal_color))
fig.add_trace(go.Bar(x=x, y=list(df_pt_all_cap_tech_allscs.Battery), name='Battery new', marker_color=battery_color))

#Remove white lines separating elements of bars
fig.update_traces(marker_line_width = 0, 
                  selector=dict(type="bar"))

fig.update_layout(barmode="relative",yaxis_title="Existing and new capacity (GW)", font = dict(size=12),
                  width=1200,height=520, plot_bgcolor='white',#showlegend=False)
                  legend=dict(yanchor="top",y=1.0,xanchor="left", x=1.00, traceorder='reversed'),
                  margin=dict(l=0,r=0,b=0,t=0))

# fig.update_layout(title_text = "A", title_x = 0, title_y=0.99, title_font=dict(size=14))

# Change grid color and axis colors
# fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
fig.update_yaxes(gridcolor='#f5f5f5') # linewidth=2, linecolor='black', 
fig.update_xaxes(tickangle=270)

fig.show()

if save_outputs == 'yes':
    fig.write_image(figpath + 'allcap_{}.png'.format(suffix))


# Plot capacity differences w.r.t. reference

In [7]:
##list of column names with technologies, exclude last column for scenario name
value_columns = list_pt_all_cap_tech[0].columns[:-1]

##deduct capacity of Reference scenario from each scenario and assign to a list
list_pt_ncap1_tech_anom = []

for i in range(len(scs)):
    anom = list_pt_all_cap_tech[i][value_columns] - df_pt_all_cap_tech_allscs[df_pt_all_cap_tech_allscs.scs == ref_scs_tick][value_columns]
    anom['scs'] = scs[i]
    
    list_pt_ncap1_tech_anom.append(anom)
    
##concat all dfs with delta capacity w.r.t. Reference scenario
df_pt_ncap1_tech_anom_allscs = pd.concat([list_pt_ncap1_tech_anom[i] for i in range(len(scs))])

#print(df_pt_ncap1_tech_anom_allscs.head())

In [8]:
##separate existing and new-built capacities of Reference scenario
base_sc = df_pt_all_cap_tech_allscs[(df_pt_all_cap_tech_allscs.scs == ref_scs_tick)].copy()# &
                                    #(df_pt_all_cap_tech_allscs.index != 2025) &
                                    #(df_pt_all_cap_tech_allscs.index != 2035) &
                                    #(df_pt_all_cap_tech_allscs.index != 2045)].copy()

## separate delta capacities of other scenarios, also exclude 2020's data as that is same across all scenarios
alt_sc = df_pt_ncap1_tech_anom_allscs[(df_pt_ncap1_tech_anom_allscs.scs != ref_scs_tick) & 
                                      (df_pt_ncap1_tech_anom_allscs.index != 2020)].copy()# &
                                      #(df_pt_ncap1_tech_anom_allscs.index != 2025) &
                                      #(df_pt_ncap1_tech_anom_allscs.index != 2035) &
                                      #(df_pt_ncap1_tech_anom_allscs.index != 2045)].copy()

##plot actual capacities of Ref. and delta capacities of other scenarios

x1 = [list(base_sc.scs),
    list(base_sc.index)]

x2 = [list(alt_sc.index),
    list(alt_sc.scs)]

fig = make_subplots(rows=1, cols=2, column_widths=[0.15, 0.85], shared_yaxes=False,horizontal_spacing=0.15)

##Reference scenario -- Existing capacities
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Coal), name='Coal existing', marker_color=coal_color, marker_pattern_shape="x",showlegend=False),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Gas), name='Gas existing', marker_color=gas_color, marker_pattern_shape="x",showlegend=False),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Hydro), name='Hydro existing', marker_color=hydro_color, marker_pattern_shape="x",showlegend=False),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Solar), name='Solar existing', marker_color=solar_color, marker_pattern_shape="x",showlegend=False),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Wind), name='Wind existing', marker_color=wind_color, marker_pattern_shape="x",showlegend=False),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Nuclear), name='Nuclear existing', marker_color=nuclear_color, marker_pattern_shape="x",showlegend=False),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Biomass), name='Biomass existing', marker_color=biomass_color, marker_pattern_shape="x",showlegend=False),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Diesel), name='Diesel existing', marker_color=diesel_color, marker_pattern_shape="x",showlegend=False),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Geothermal), name='Geothermal existing', marker_color=geothermal_color, marker_pattern_shape="x",showlegend=False),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(np.zeros(len(base_sc.Geothermal))), name='Existing', marker_color='white', marker_pattern_shape="x",showlegend=True),row=1,col=1)


##Reference scenario -- New capacities
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Coal_new), name='Coal', marker_color=coal_color),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Coal_CCS), name='Coal CCS', marker_color=coal_color, marker_pattern_shape="."),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Gas_new), name='Gas', marker_color=gas_color),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Gas_CCS), name='Gas CCS', marker_color=gas_color, marker_pattern_shape="."),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Hydro_new), name='Hydro', marker_color=hydro_color),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Solar_new), name='Solar', marker_color=solar_color),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Wind_new), name='Wind', marker_color=wind_color),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Nuclear_new), name='Nuclear', marker_color=nuclear_color),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Biomass_new), name='Biomass', marker_color=biomass_color),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Biomass_CCS), name='Biomass CCS', marker_color=biomass_color, marker_pattern_shape="."),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Diesel_new), name='Diesel', marker_color=diesel_color),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Diesel_CCS), name='Diesel CCS', marker_color=diesel_color, marker_pattern_shape="."),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Geothermal_new), name='Geothermal', marker_color=geothermal_color),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Battery), name='Battery', marker_color=battery_color),row=1,col=1)


###Alternative scenarios -- Existing capacities (diff from ref.)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Coal), name='Coal existing', marker_color=coal_color, marker_pattern_shape="x",showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Gas), name='Gas existing', marker_color=gas_color, marker_pattern_shape="x",showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Hydro), name='Hydro existing', marker_color=hydro_color, marker_pattern_shape="x",showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Solar), name='Solar existing', marker_color=solar_color, marker_pattern_shape="x",showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Wind), name='Wind existing', marker_color=wind_color, marker_pattern_shape="x",showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Nuclear), name='Nuclear existing', marker_color=nuclear_color, marker_pattern_shape="x",showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Biomass), name='Biomass existing', marker_color=biomass_color, marker_pattern_shape="x",showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Diesel), name='Diesel existing', marker_color=diesel_color, marker_pattern_shape="x",showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Geothermal), name='Geothermal existing', marker_color=geothermal_color, marker_pattern_shape="x",showlegend=False),row=1,col=2)


###Alternative scenarios -- New capacities (diff from ref.)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Coal_new), name='Coal new', marker_color=coal_color,showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Coal_CCS), name='Coal CCS new', marker_color=coal_color, marker_pattern_shape=".",showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Gas_new), name='Gas new', marker_color=gas_color,showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Gas_CCS), name='Gas CCS new', marker_color=gas_color, marker_pattern_shape=".",showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Hydro_new), name='Hydro new', marker_color=hydro_color,showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Solar_new), name='Solar new', marker_color=solar_color,showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Wind_new), name='Wind new', marker_color=wind_color,showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Nuclear_new), name='Nuclear new', marker_color=nuclear_color,showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Biomass_new), name='Biomass new', marker_color=biomass_color,showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Biomass_CCS), name='Biomass CCS new', marker_color=biomass_color, marker_pattern_shape=".",showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Diesel_new), name='Diesel new', marker_color=diesel_color,showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Diesel_CCS), name='Diesel CCS new', marker_color=diesel_color, marker_pattern_shape=".",showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Geothermal_new), name='Geothermal new', marker_color=geothermal_color,showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Battery), name='Battery new', marker_color=battery_color,showlegend=False),row=1,col=2)


#Remove white lines separating elements of bars
fig.update_traces(marker_line_width = 0, 
                  selector=dict(type="bar"))

fig.update_layout(barmode="relative", font = dict(size=12),
                  yaxis=dict(title = "Generation capacity (GW)"), 
                  yaxis2=dict(title = "Change in gen. capacity (GW) <br> w.r.t. Reference"), 
                  width=910, height=400, plot_bgcolor='white',legend_traceorder="reversed",#showlegend=False)
                  legend=dict(yanchor="top",y=0.85,xanchor="left", x=1.00), margin=dict(l=10,r=10,b=10,t=10))

# Change grid color and axis colors
# fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
fig.update_yaxes(gridcolor='#f5f5f5') # linewidth=2, linecolor='black', 

fig.show()

if save_outputs == 'yes':
    fig.write_image(figpath + 'allcap_diff_byperiod_{}.png'.format(suffix))

# By scs and with only existing in legend

In [9]:
##separate existing and new-built capacities of Reference scenario
base_sc = df_pt_all_cap_tech_allscs[(df_pt_all_cap_tech_allscs.scs == ref_scs_tick)].copy()# &
                                    #(df_pt_all_cap_tech_allscs.index != 2025) &
                                    #(df_pt_all_cap_tech_allscs.index != 2035) &
                                    #(df_pt_all_cap_tech_allscs.index != 2045)].copy()

## separate delta capacities of other scenarios, also exclude 2020's data as that is same across all scenarios
alt_sc = df_pt_ncap1_tech_anom_allscs[(df_pt_ncap1_tech_anom_allscs.scs != ref_scs_tick) & 
                                      (df_pt_ncap1_tech_anom_allscs.index != 2020)].copy()# &
                                      #(df_pt_ncap1_tech_anom_allscs.index != 2025) &
                                      #(df_pt_ncap1_tech_anom_allscs.index != 2035) &
                                      #(df_pt_ncap1_tech_anom_allscs.index != 2045)].copy()

##plot actual capacities of Ref. and delta capacities of other scenarios

x1 = [list(base_sc.scs),
    list(base_sc.index)]

x2 = [list(alt_sc.scs),
    list(alt_sc.index)]

fig = make_subplots(rows=1, cols=2, column_widths=[0.15, 0.85], shared_yaxes=False, horizontal_spacing=0.12)

##Reference scenario -- Existing capacities
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Coal), name='Coal existing', marker_color=coal_color, marker_pattern_shape="x",showlegend=False),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Gas), name='Gas existing', marker_color=gas_color, marker_pattern_shape="x",showlegend=False),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Hydro), name='Hydro existing', marker_color=hydro_color, marker_pattern_shape="x",showlegend=False),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Solar), name='Solar existing', marker_color=solar_color, marker_pattern_shape="x",showlegend=False),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Wind), name='Wind existing', marker_color=wind_color, marker_pattern_shape="x",showlegend=False),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Nuclear), name='Nuclear existing', marker_color=nuclear_color, marker_pattern_shape="x",showlegend=False),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Biomass), name='Biomass existing', marker_color=biomass_color, marker_pattern_shape="x",showlegend=False),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Diesel), name='Diesel existing', marker_color=diesel_color, marker_pattern_shape="x",showlegend=False),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Geothermal), name='Geothermal existing', marker_color=geothermal_color, marker_pattern_shape="x",showlegend=False),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(np.zeros(len(base_sc.Geothermal))), name='Existing', marker_color='white', marker_pattern_shape="x",showlegend=True),row=1,col=1)


##Reference scenario -- New capacities
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Coal_new), name='Coal', marker_color=coal_color,showlegend=False),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Coal_CCS), name='Coal CCS', marker_color=coal_color, marker_pattern_shape=".",showlegend=False),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Gas_new), name='Gas', marker_color=gas_color,showlegend=False),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Gas_CCS), name='Gas CCS', marker_color=gas_color, marker_pattern_shape=".",showlegend=False),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Hydro_new), name='Hydro', marker_color=hydro_color,showlegend=False),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Solar_new), name='Solar', marker_color=solar_color,showlegend=False),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Wind_new), name='Wind', marker_color=wind_color,showlegend=False),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Nuclear_new), name='Nuclear', marker_color=nuclear_color,showlegend=False),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Biomass_new), name='Biomass', marker_color=biomass_color,showlegend=False),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Biomass_CCS), name='Biomass CCS', marker_color=biomass_color, marker_pattern_shape=".",showlegend=False),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Diesel_new), name='Diesel', marker_color=diesel_color,showlegend=False),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Diesel_CCS), name='Diesel CCS', marker_color=diesel_color, marker_pattern_shape=".",showlegend=False),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Geothermal_new), name='Geothermal', marker_color=geothermal_color,showlegend=False),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Battery), name='Battery', marker_color=battery_color,showlegend=False),row=1,col=1)


###Alternative scenarios -- Existing capacities (diff from ref.)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Coal), name='Coal existing', marker_color=coal_color, marker_pattern_shape="x",showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Gas), name='Gas existing', marker_color=gas_color, marker_pattern_shape="x",showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Hydro), name='Hydro existing', marker_color=hydro_color, marker_pattern_shape="x",showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Solar), name='Solar existing', marker_color=solar_color, marker_pattern_shape="x",showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Wind), name='Wind existing', marker_color=wind_color, marker_pattern_shape="x",showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Nuclear), name='Nuclear existing', marker_color=nuclear_color, marker_pattern_shape="x",showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Biomass), name='Biomass existing', marker_color=biomass_color, marker_pattern_shape="x",showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Diesel), name='Diesel existing', marker_color=diesel_color, marker_pattern_shape="x",showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Geothermal), name='Geothermal existing', marker_color=geothermal_color, marker_pattern_shape="x",showlegend=False),row=1,col=2)


###Alternative scenarios -- New capacities (diff from ref.)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Coal_new), name='Coal new', marker_color=coal_color,showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Coal_CCS), name='Coal CCS new', marker_color=coal_color, marker_pattern_shape=".",showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Gas_new), name='Gas new', marker_color=gas_color,showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Gas_CCS), name='Gas CCS new', marker_color=gas_color, marker_pattern_shape=".",showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Hydro_new), name='Hydro new', marker_color=hydro_color,showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Solar_new), name='Solar new', marker_color=solar_color,showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Wind_new), name='Wind new', marker_color=wind_color,showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Nuclear_new), name='Nuclear new', marker_color=nuclear_color,showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Biomass_new), name='Biomass new', marker_color=biomass_color,showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Biomass_CCS), name='Biomass CCS new', marker_color=biomass_color, marker_pattern_shape=".",showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Diesel_new), name='Diesel new', marker_color=diesel_color,showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Diesel_CCS), name='Diesel CCS new', marker_color=diesel_color, marker_pattern_shape=".",showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Geothermal_new), name='Geothermal new', marker_color=geothermal_color,showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Battery), name='Battery new', marker_color=battery_color,showlegend=False),row=1,col=2)


#Remove white lines separating elements of bars
fig.update_traces(marker_line_width = 0, 
                  selector=dict(type="bar"))

fig.update_layout(barmode="relative", font = dict(size=12),
                  yaxis=dict(title = "Capacity (GW)"), 
                  yaxis2=dict(title = "Capacity difference (GW)"), 
                  width=920, height=400, plot_bgcolor='white',legend_traceorder="reversed",#showlegend=False)
                  legend=dict(yanchor="top",y=0.5,xanchor="left", x=1.00), margin=dict(l=10,r=10,b=10,t=10))

fig.update_layout(title_text = "(a)", title_x = 0.5, title_y=0.99, title_font=dict(size=14))

# Change grid color and axis colors
# fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
fig.update_yaxes(gridcolor='#f5f5f5') # linewidth=2, linecolor='black', 

fig.show()

if save_outputs == 'yes':
    fig.write_image(figpath + 'allcap_diff_byscs_{}.png'.format(suffix))

# TWh

In [10]:
list_pt_gen_loss_curt = []

for i in range(len(scs)): #len(scs)
    ###Generation-TWh 2020-2050
    alldisp1 = pd.read_csv(respath+''+str(scs[i])+'/results/dispatch_all.csv')    
    alldisp1['twh'] = alldisp1['timepoint_weight'] * alldisp1['power_mw']/(10**6)    
    alldisp1_tec_twh_nstor = alldisp1[(alldisp1.operational_type!='stor')].groupby(['period','technology'])['twh'].sum() #& (alldisp1.period!=2045)
    alldisp1_tec_twh_stor = alldisp1[(alldisp1.operational_type=='stor') & (alldisp1.twh<=0)].groupby(['period','technology'])['twh'].sum() #& (alldisp1.period!=2045)
    alldisp1_tec_twh_ns = pd.concat([alldisp1_tec_twh_nstor,alldisp1_tec_twh_stor]).reset_index()
    
    ###Trans_Loss 2020-2050
    trans = pd.read_csv(respath+''+str(scs[i])+'/results/transmission_operations.csv')
    trans['twh'] = (trans['transmission_losses_lz_from']+trans['transmission_losses_lz_to'])*trans['timepoint_weight']/(10**6)
    df_trans_loss = pd.DataFrame([trans.groupby(['period'])['twh'].sum()*(-1)]).transpose() #[(trans.period!=2045)]
    df_trans_loss['technology']='Trans_loss'
    df_trans_loss = df_trans_loss.reset_index()

    ###Curtailment-VRE 2020-2050
    disp_re = pd.read_csv(respath+''+str(scs[i])+'/results/dispatch_variable.csv')
    disp_re['twh'] = disp_re['total_curtailment_mw']*disp_re['timepoint_weight']/(10**6)
    curt_re = disp_re.groupby(['period'])['twh'].sum() #[(disp_re.period!=2045)]
    curt_re = curt_re.reset_index()
    curt_re['technology']='curt_vre'

    ###Curtailment-Hydro 2020-2050
    disp_hydro = pd.read_csv(respath+''+str(scs[i])+'/results/dispatch_gen_hydro.csv')
    disp_hydro['twh'] = disp_hydro['scheduled_curtailment_mw']*disp_hydro['timepoint_weight']/(10**6)
    curt_hydro = disp_hydro.groupby(['period'])['twh'].sum() #[(disp_hydro.period!=2045)]
    curt_hydro = curt_hydro.reset_index()
    curt_hydro['technology']='curt_hydro'

    ###Concat Gen_TWh, Trans_loss, and Curtailments
    df_gen_loss_curt = pd.concat([alldisp1_tec_twh_ns, df_trans_loss, curt_re, curt_hydro],axis=0)
    df_gen_loss_curt.set_index('period').sort_index()

    ###rename tech for plotting
    df_gen_loss_curt.loc[df_gen_loss_curt['technology'] == 'Coal_Sub','technology'] = 'Coal'
    df_gen_loss_curt.loc[df_gen_loss_curt['technology'] == 'Coal_Sup','technology'] = 'Coal'
    df_gen_loss_curt.loc[df_gen_loss_curt['technology'] == 'Coal_IGCC','technology'] = 'Coal'
    df_gen_loss_curt.loc[df_gen_loss_curt['technology'] == 'Gas_CCGT','technology'] = 'Gas'
    df_gen_loss_curt.loc[df_gen_loss_curt['technology'] == 'Gas_OCGT','technology'] = 'Gas'
    df_gen_loss_curt.loc[df_gen_loss_curt['technology'] == 'Gas_ICE','technology'] = 'Gas'
    df_gen_loss_curt.loc[df_gen_loss_curt['technology'] == 'Diesel','technology'] = 'Diesel'
    df_gen_loss_curt.loc[df_gen_loss_curt['technology'] == 'Oil','technology'] = 'Diesel'
    df_gen_loss_curt.loc[df_gen_loss_curt['technology'] == 'Geothermal','technology'] = 'Geothermal'
    df_gen_loss_curt.loc[df_gen_loss_curt['technology'] == 'Biomass','technology'] = 'Biomass'
    df_gen_loss_curt.loc[df_gen_loss_curt['technology'] == 'SolarPV','technology'] = 'Solar'
    df_gen_loss_curt.loc[df_gen_loss_curt['technology'] == 'curt_vre','technology'] = 'Curtailment'
    df_gen_loss_curt.loc[df_gen_loss_curt['technology'] == 'curt_hydro','technology'] = 'Curtailment'

    ##sum of twh by renamed tech and period
    df_gen_loss_curt_reclass = df_gen_loss_curt.groupby(['period','technology'])['twh'].sum().reset_index()

    ###Creat pivot table
    pt_gen_loss_curt = df_gen_loss_curt_reclass.pivot(index='period', columns='technology', values='twh')

    ###assign scenario name in a column
    pt_gen_loss_curt['scs'] = scs[i]

    ###Append to the pivot table to list
    list_pt_gen_loss_curt.append(pt_gen_loss_curt)

len(list_pt_gen_loss_curt)

6

# Plot TWh for all periods

In [11]:
### Concat pivot tables of all scenarios
df_pt_gen_loss_curt_allscs = pd.concat([list_pt_gen_loss_curt[i] for i in range(len(scs))])

# Shortening Scenario Names (if needed)
for i in range(len(df_pt_all_cap_tech_allscs)):
    df_pt_gen_loss_curt_allscs.iloc[i,-1] = df_pt_gen_loss_curt_allscs.iloc[i,-1][4:]
ref_scs_tick = ref[4:]

# df_pt_gen_loss_curt_allscs = df_pt_gen_loss_curt_allscs[(df_pt_gen_loss_curt_allscs.index==2020) & 
#                                                         (df_pt_gen_loss_curt_allscs.scs==ref_scs_tick) |
#                                                         (df_pt_gen_loss_curt_allscs.index==2025) |
#                                                         (df_pt_gen_loss_curt_allscs.index==2030) |
#                                                         (df_pt_gen_loss_curt_allscs.index==2035) |
#                                                         (df_pt_gen_loss_curt_allscs.index==2040) |
#                                                         (df_pt_gen_loss_curt_allscs.index==2045) |
#                                                         (df_pt_gen_loss_curt_allscs.index==2050)].copy()

In [12]:
#df_pt_gen_loss_curt_allscs.to_csv('gglac_gen_mix_sw.csv')
df_pt_gen_loss_curt_allscs

technology  Battery    Biomass  Biomass_CCS  Biomass_IGCC       Coal  \
period                                                                 
2020            0.0  67.387263          0.0           0.0  19.387551   
2025            0.0  62.859267          0.0           0.0  17.069709   
2030            0.0  70.649368          0.0           0.0  14.158862   
2035            0.0  80.473355          0.0           0.0   7.675213   
2040            0.0  83.487809          0.0           0.0   4.085590   
2045            0.0  79.791384          0.0           0.0   2.860806   
2050            0.0  80.081995          0.0           0.0   2.643840   
2020            0.0  67.387263          0.0           0.0  19.387551   
2025            0.0  48.269846          0.0           0.0  16.842208   
2030            0.0  58.578357          0.0           0.0  16.726934   
2035            0.0  81.048256          0.0           0.0   7.712246   
2040            0.0  83.115388          0.0           0.0   4.150045   
2045            0.0  80.610575          0.0           0.0   2.643840   
2050            0.0  81.443942          0.0           0.0   2.530650   
2020            0.0  67.387263          0.0           0.0  19.387551   
2025            0.0  48.764751          0.0           0.0  17.275314   
2030            0.0  63.978725          0.0           0.0  20.123907   
2035            0.0  79.555291          0.0           0.0   8.040507   
2040            0.0  83.397240          0.0           0.0   4.085370   
2045            0.0  80.769353          0.0           0.0   2.643840   
2050            0.0  81.061439          0.0           0.0   0.584137   
2020            0.0  67.387263          0.0           0.0  19.387551   
2025            0.0  65.633247          0.0           0.0  17.072168   
2030            0.0  76.302304          0.0           0.0  16.134348   
2035            0.0  81.128368          0.0           0.0   9.186667   
2040            0.0  84.044308          0.0           0.0   8.408353   
2045            0.0  84.853010          0.0           0.0   3.555893   
2050            0.0  88.681292          0.0           0.0   4.049419   
2020            0.0  67.387263          0.0           0.0  19.387551   
2025            0.0  48.269846          0.0           0.0  16.842208   
2030            0.0  64.913091          0.0           0.0  18.439964   
2035            0.0  81.611200          0.0           0.0   9.665776   
2040            0.0  84.254547          0.0           0.0   6.387933   
2045            0.0  88.595829          0.0           0.0   6.363845   
2050            0.0  88.825596          0.0           0.0   8.990292   
2020            0.0  67.387263          0.0           0.0  19.387551   
2025            0.0  48.764751          0.0           0.0  17.275314   
2030            0.0  65.882465          0.0           0.0  17.760202   
2035            0.0  79.220269          0.0           0.0   9.761641   
2040            0.0  85.705170          0.0           0.0   6.472883   
2045            0.0  88.913225          0.0           0.0   7.496433   
2050            0.0  88.990934          0.0           0.0   2.871409   

technology  Coal_CCS   Curtailment    Diesel  Diesel_CCS         Gas  Gas_CCS  \
period                                                                          
2020             0.0  6.821386e+00  9.004958         0.0  153.690754      0.0   
2025             0.0  7.548978e+00  0.000000         0.0  198.307612      0.0   
2030             0.0  5.907798e+00  0.000000         0.0  203.031746      0.0   
2035             0.0  3.072969e-14  0.000000         0.0  326.616334      0.0   
2040             0.0  1.711608e+00  0.000000         0.0  428.086497      0.0   
2045             0.0  3.420369e-14  0.000000         0.0  548.620325      0.0   
2050             0.0  3.954267e-01  0.000000         0.0  631.856400      0.0   
2020             0.0  6.821386e+00  9.004958         0.0  153.690754      0.0   
2025             0.0 

In [13]:
# # get gcam gen mixes here (SYSTEM-WIDE)

# genpath='/cluster/tufts/lamontagnelab/jwesse03/GCAM6/gcam-core/output/paper1_LAC_basexdb/paper1_queries/batch_query_files/python_scripts/processed_data/gen_tech_by_region/'
# gensa = pd.read_csv(genpath+'gen_South America_Southern.csv')
# genar = pd.read_csv(genpath+'gen_Argentina.csv')
# genbr = pd.read_csv(genpath+'gen_Brazil.csv')
# genuy = pd.read_csv(genpath+'gen_Uruguay.csv')
# genar = genar[(genar.scenario.isin(['jaw_LAC_s2e0mcif0c0i0b0n0l0t0h0','jaw_LAC_s2e1mcif0c0i0b0n0l0t0h0']))&\
#           (genar.year.isin([2020,2025,2030,2035,2040,2045,2050]))].copy(deep=True)
# genbr = genbr[(genbr.scenario.isin(['jaw_LAC_s2e0mcif0c0i0b0n0l0t0h0','jaw_LAC_s2e1mcif0c0i0b0n0l0t0h0']))&\
#           (genbr.year.isin([2020,2025,2030,2035,2040,2045,2050]))].copy(deep=True)
# genuy = genuy[(genuy.scenario.isin(['jaw_LAC_s2e0mcif0c0i0b0n0l0t0h0','jaw_LAC_s2e1mcif0c0i0b0n0l0t0h0']))&\
#           (genuy.year.isin([2020,2025,2030,2035,2040,2045,2050]))].copy(deep=True)
# gensa = gensa[(gensa.scenario.isin(['jaw_LAC_s2e0mcif0c0i0b0n0l0t0h0','jaw_LAC_s2e1mcif0c0i0b0n0l0t0h0']))&\
#           (gensa.year.isin([2020,2025,2030,2035,2040,2045,2050]))].copy(deep=True)

# gensa['chl_frac']=gensa['year'].map({2020:0.344442,2025:0.335099,2030:0.32497,2035:0.31538,2040:0.306476,2045:0.298366,2050:0.289936})
# gensa['pry_frac']=gensa['year'].map({2020:0.088057,2025:0.090023,2030:0.091866,2035:0.093712,2040:0.095528,2045:0.097295,2050:0.099043})
# gensa['chl'] = gensa['value']*gensa['chl_frac']
# gencl = gensa[['Units','scenario','region','subsector','year','chl']].copy(deep=True).rename(columns={'chl':'value'})
# gencl['region'] = 'Chile'
# gensa['pry'] = gensa['value']*gensa['pry_frac']
# genpy = gensa[['Units','scenario','region','subsector','year','pry']].copy(deep=True).rename(columns={'pry':'value'})
# genpy['region'] = 'Paraguay'

# genmer = pd.concat([genar,genbr,gencl,genpy,genuy]).reset_index(drop=True)
# genmer['subsector'] = genmer['subsector'].map({'biomass':'Biomass','coal':'Coal','gas':'Gas','geothermal':'Geothermal','hydro':'Hydro',
#                                                'nuclear':'Nuclear','refined liquids':'Diesel','solar':'Solar','wind':'Wind'})
# genmer = genmer.rename(columns={'region':'load_zone','subsector':'technology','year':'period','value':'twh','scenario':'scs'}).drop(columns='Units')
# genmer['twh'] *= 277.777777778

# genmer.to_csv('gcam_gen_mix.csv', index=False)
# genmer

genmer = pd.read_csv('gcam_gen_mix.csv')
genmer

genmer1 = genmer[genmer.scs=='jaw_LAC_s2e0mcif0c0i0b0n0l0t0h0'].copy(deep=True)
ptgenmer1 = genmer1.pivot(index=['period','load_zone'], columns='technology', values='twh')
ptgenmer1['scs'] = 'gcam_ref'

genmer2 = genmer[genmer.scs=='jaw_LAC_s2e1mcif0c0i0b0n0l0t0h0'].copy(deep=True)
ptgenmer2 = genmer2.pivot(index=['period','load_zone'], columns='technology', values='twh')
ptgenmer2['scs'] = 'gcam_decarb'

ptgenmer = pd.concat([ptgenmer1.reset_index(),ptgenmer2.reset_index()])
ptgenmer

technology  period  load_zone    Biomass       Coal     Diesel         Gas  \
0             2020  Argentina   2.125469   3.864806  16.842686   76.352472   
1             2020     Brazil   0.731839  21.165667  17.371056   75.712778   
2             2020      Chile   1.103056  12.300311   3.864812   17.589179   
3             2020   Paraguay   0.281998   3.144589   0.988044    4.496694   
4             2020    Uruguay   0.316114   0.021502   1.158993    0.052623   
..             ...        ...        ...        ...        ...         ...   
30            2050  Argentina   7.500438   2.357878  23.070367  160.445528   
31            2050     Brazil   7.442153  26.576500  47.725542  367.669694   
32            2050      Chile   2.605027  17.444483   9.479295   48.471533   
33            2050   Paraguay   0.889885   5.959087   3.238155   16.558020   
34            2050    Uruguay  11.660602   0.056817   2.238833    0.276656   

technology  Geothermal       Hydro    Nuclear       Solar        Wind  \
0             0.531978   39.211389   7.208067    1.309993    2.326108   
1             0.731683  371.588889  14.274569    4.723198   29.622726   
2             0.157389   45.622300   0.000000    1.581324    1.733544   
3             0.040237   11.663394   0.000000    0.404267    0.443183   
4             0.124609    8.266000   0.000000    0.691523    2.702952   
..                 ...         ...        ...         ...         ...   
30           24.931361   46.246667  21.876806   84.291762  128.821014   
31           71.286944  442.672222  53.407694  315.076124  415.964844   
32            8.058771   61.989928   3.697377   41.074497   37.312737   
33            2.752900   21.175944   1.263035   14.031170   12.746142   
34            2.211903    8.266000   0.786467    7.612540    7.054559   

technology          scs  
0              gcam_ref  
1              gcam_ref  
2              gcam_ref  
3              gcam_ref  
4              gcam_ref  
..                  ...  
30          gcam_decarb  
31          gcam_decarb  
32          gcam_decarb  
33          gcam_decarb  
34          gcam_decarb  

[70 rows x 12 columns]

In [14]:
# gcam_gen_sw = ptgenmer.groupby(['period','scs']).sum().reset_index()
# gcam_gen_sw

In [15]:
# gp_gen = pd.read_csv('gglac_gen_mix_sw.csv')
# gp_gen = gp_gen[['period','Biomass','Coal','Diesel','Gas','Geothermal','Hydro',
#                  'Nuclear','Solar','Wind','scs','Trans_loss','Curtailment']]
# gp_gen

In [16]:
### final data read-in ###

# gendf = pd.concat([gcam_gen_sw,gp_gen]).sort_values(by=['scs','period']).set_index('period')
# gendf = gendf.fillna(0)
# gendf.to_csv('combined_gen_mix_gp2gc.csv')

gendf = pd.read_csv('combined_gen_mix_gp2gc.csv',index_col='period')
gendf

scs    Biomass        Coal     Diesel         Gas  \
period                                                                
2020      gcam_decarb   4.192890   31.358867  35.102108  144.121050   
2025      gcam_decarb   7.982362   43.931333  48.957839  204.925012   
2030      gcam_decarb  14.171116   60.060811  59.189765  295.800740   
2035      gcam_decarb  18.287103   69.698967  69.682371  377.262508   
2040      gcam_decarb  21.397330   78.252443  77.830853  472.372967   
2045      gcam_decarb  24.488523   64.207008  81.192220  529.498327   
2050      gcam_decarb  30.098104   52.394764  85.752192  593.421431   
2020         gcam_ref   4.558476   40.496874  40.225591  174.203747   
2025         gcam_ref   7.280095   54.255408  52.213692  244.139527   
2030         gcam_ref  10.845887   72.978324  64.189716  349.643008   
2035         gcam_ref  14.732174   92.872690  76.982054  471.597983   
2040         gcam_ref  18.439730  112.425546  83.828853  595.650910   
2045         gcam_ref  21.124002  129.288338  82.743372  710.548098   
2050         gcam_ref  22.963858  142.327166  81.502657  798.791746   
2020    gg_ref_adj_l1  67.387263   19.387551   9.004958  153.690754   
2025    gg_ref_adj_l1  62.859267   17.069709   0.000000  198.307612   
2030    gg_ref_adj_l1  70.649368   14.158862   0.000000  203.031746   
2035    gg_ref_adj_l1  80.473355    7.675213   0.000000  326.616334   
2040    gg_ref_adj_l1  83.487809    4.085590   0.000000  428.086497   
2045    gg_ref_adj_l1  79.791384    2.860806   0.000000  548.620325   
2050    gg_ref_adj_l1  80.081995    2.643840   0.000000  631.856400   
2020    gg_ref_adj_l2  67.387263   19.387551   9.004958  153.690754   
2025    gg_ref_adj_l2  48.269846   16.842208   0.000000  159.871613   
2030    gg_ref_adj_l2  58.578357   16.726934   0.000000  179.722136   
2035    gg_ref_adj_l2  81.048256    7.712246   0.000000  349.236168   
2040    gg_ref_adj_l2  83.115388    4.150045   0.000000  508.008194   
2045    gg_ref_adj_l2  80.610575    2.643840   0.000000  595.214234   
2050    gg_ref_adj_l2  81.443942    2.530650   0.000000  639.702387   
2020    gg_ref_adj_l3  67.387263   19.387551   9.004958  153.690754   
2025    gg_ref_adj_l3  48.764751   17.275314   0.000000  171.396977   
2030    gg_ref_adj_l3  63.978725   20.123907   0.000000  175.225901   
2035    gg_ref_adj_l3  79.555291    8.040507   0.000000  336.810263   
2040    gg_ref_adj_l3  83.397240    4.085370   0.000000  501.706329   
2045    gg_ref_adj_l3  80.769353    2.643840   0.000000  578.290593   
2050    gg_ref_adj_l3  81.061439    0.584137   0.000000  630.095518   
2020        gg_ref_l1  67.387263   19.387551   9.004958  153.690754   
2025        gg_ref_l1  65.633247   17.072168   0.000000  202.373740   
2030        gg_ref_l1  76.302304   16.134348   0.000000  212.383544   
2035        gg_ref_l1  81.128368    9.186667   0.000000  337.891779   
2040        gg_ref_l1  84.044308    8.408353   0.000000  447.459953   
2045        gg_ref_l1  84.853010    3.555893   0.000000  615.003265   
2050        gg_ref_l1  88.681292    4.049419   0.000000  691.593198   
2020        gg_ref_l2  67.387263   19.387551   9.004958  153.690754   
2025        gg_ref_l2  48.269846   16.842208   0.000000  159.871613   
2030        gg_ref_l2  64.913091   18.439964   0.000000  188.419531   
2035        gg_ref_l2  81.611200    9.665776   0.000000  364.228329   
2040        gg_ref_l2  84.254547    6.387933   0.000000  540.970054   
2045        gg_ref_l2  88.595829    6.363845   0.000000  696.572847   
2050        gg_ref_l2  88.825596    8.990292   0.000000  739.716556   
2020        gg_ref_l3  67.387263   19.387551   9.004958  153.690754   
2025        gg_ref_l3  48.764751   17.275314   0.000000  171.396977   
2030        gg_ref_l3  65.882465   17.760202   0.000000  188.037859   
2035        gg_ref_l3  79.220269    9.761641   0.000000  347.723505   
2040        gg_ref_l3  85.705170    6.472883   0.000000  542.114159   
2045        gg_ref_l3  88.913225    7.4964

fraction of south america southern GDP attributed to Chile:
      chl_frac
{2020:0.344442,
2025:0.335099,
2030:0.32497,
2035:0.31538,
2040:0.306476,
2045:0.298366,
2050:0.289936}

fraction of south america southern POP attributed to Paraguay:
      pry_frac
{2020:0.088057,
2025:0.090023,
2030:0.091866,
2035:0.093712,
2040:0.095528,
2045:0.097295,
2050:0.099043}

In [17]:
import plotly.graph_objects as go

x = [list(gendf.index),
     list(gendf.scs)]

fig = go.Figure(go.Bar(x=x,y=list(gendf.Trans_loss), name='Transmission <br>loss', marker_color=trans_color))
fig.add_trace(go.Bar(x=x,y=list(gendf.Coal), name='Coal', marker_color=coal_color))
#fig.add_trace(go.Bar(x=x,y=list(gendf.Coal_CCS), name='Coal CCS', marker_color=coal_color, marker_pattern_shape="."))
fig.add_trace(go.Bar(x=x,y=list(gendf.Gas), name='Gas', marker_color=gas_color))
#fig.add_trace(go.Bar(x=x,y=list(gendf.Gas_CCS), name='Gas CCS', marker_color=gas_color, marker_pattern_shape="."))
fig.add_trace(go.Bar(x=x,y=list(gendf.Nuclear), name='Nuclear', marker_color=nuclear_color))
fig.add_trace(go.Bar(x=x,y=list(gendf.Hydro), name='Hydro', marker_color=hydro_color))
fig.add_trace(go.Bar(x=x,y=list(gendf.Solar), name='Solar', marker_color=solar_color))
fig.add_trace(go.Bar(x=x,y=list(gendf.Wind), name='Wind', marker_color=wind_color))
fig.add_trace(go.Bar(x=x,y=list(gendf.Biomass), name='Biomass', marker_color=biomass_color))
#fig.add_trace(go.Bar(x=x,y=list(gendf.Biomass_CCS), name='Biomass CCS', marker_color=biomass_color, marker_pattern_shape="."))
fig.add_trace(go.Bar(x=x,y=list(gendf.Diesel), name='Diesel', marker_color=diesel_color))
#fig.add_trace(go.Bar(x=x,y=list(gendf.Diesel_CCS), name='Diesel CCS', marker_color=diesel_color, marker_pattern_shape="."))
fig.add_trace(go.Bar(x=x,y=list(gendf.Geothermal), name='Geothermal', marker_color=geothermal_color))
fig.add_trace(go.Bar(x=x,y=list(gendf.Curtailment), name='Curtailment', marker_color=curtailment_color))

#Remove white lines separating elements of bars
fig.update_traces(marker_line_width = 0,
                  selector=dict(type="bar"))

fig.update_layout(barmode="relative",yaxis_title="Electricity generation (TWh)", font = dict(size=13),
                  width=1300, height=500, plot_bgcolor='white',#showlegend=False)
                  legend=dict(yanchor="top",y=0.91,xanchor="left", x=1.00, traceorder='reversed'), 
                  margin=dict(l=0,r=0,b=0,t=0))

# fig.update_layout(title_text = "B", title_x = 0, title_y=0.99, title_font=dict(size=14))

# Change grid color and axis colors
# fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
fig.update_yaxes(gridcolor='#f5f5f5') # linewidth=2, linecolor='black', 
fig.update_xaxes(tickangle=270)

fig.show()

if save_outputs == 'yes':
    fig.write_image(figpath + 'generation_{}.png'.format(suffix), scale=4)  #"scale" increases resolution

In [18]:
# GCAM comparison of elec gen and elec consumption

genpath='diagnostic_data/'
gcam_r = ['Argentina','Brazil','Uruguay','South America_Southern']
fe0 = pd.read_csv(genpath+'final_energy_consumption_by_sector_and_fuel-jaw_LAC_s2e0mcif0c0i0b0n0l0t0h0.csv')
fe1 = pd.read_csv(genpath+'final_energy_consumption_by_sector_and_fuel-jaw_LAC_s2e1mcif0c0i0b0n0l0t0h0.csv')
fe = pd.concat([fe0,fe1])
gen0 = pd.read_csv(genpath+'elec_gen_by_gen_tech-jaw_LAC_s2e0mcif0c0i0b0n0l0t0h0.csv')
gen1 = pd.read_csv(genpath+'elec_gen_by_gen_tech-jaw_LAC_s2e1mcif0c0i0b0n0l0t0h0.csv')
gen = pd.concat([gen0,gen1])
con0 = pd.read_csv(genpath+'elec_consumption_by_demand_sector-jaw_LAC_s2e0mcif0c0i0b0n0l0t0h0.csv')
con1 = pd.read_csv(genpath+'elec_consumption_by_demand_sector-jaw_LAC_s2e1mcif0c0i0b0n0l0t0h0.csv')
con = pd.concat([con0,con1])
gen = gen[(gen.year.isin([2020,2025,2030,2035,2040,2045,2050]))&(gen.region.isin(gcam_r))&(gen.output=='electricity')].copy(deep=True)\
            .groupby(['scenario','region','year','subsector']).sum().reset_index()
gen['subsector']=gen['subsector'].map({'biomass':'Biomass','coal':'Coal','gas':'Gas','geothermal':'Geothermal','hydro':'Hydro',
                            'nuclear':'Nuclear','refined liquids':'Diesel','solar':'Solar','wind':'Wind','rooftop_pv':'Solar'})
gen = gen.groupby(['scenario','region','year']).sum().reset_index()
con = con[(con.year.isin([2020,2025,2030,2035,2040,2045,2050]))&(con.region.isin(gcam_r))].copy(deep=True)\
            .groupby(['scenario','region','year']).sum().reset_index()
fe=fe[(fe.year.isin([2020,2025,2030,2035,2040,2045,2050]))&(fe.region.isin(gcam_r))&(fe.input=='electricity')].copy(deep=True)\
            .groupby(['scenario','region','year']).sum().reset_index()

gen = gen.rename(columns={'region':'load_zone','subsector':'technology','year':'period','value':'twh','scenario':'scs'})
gen['twh'] *= 277.777777778
gen['technology'] = 'elec_gen'
con = con.rename(columns={'region':'load_zone','year':'period','value':'twh','scenario':'scs'})
con['technology'] = 'elec_consumption'
con['twh'] *= 277.777777778
fe = fe.rename(columns={'region':'load_zone','year':'period','value':'twh','scenario':'scs'})
fe['technology'] = 'fe_elec_cons'
fe['twh'] *= 277.777777778

df1 = pd.concat([gen,con,fe])
df1['scs'] = df1['scs'].map({'jaw_LAC_s2e0mcif0c0i0b0n0l0t0h0':'gcam_ref','jaw_LAC_s2e1mcif0c0i0b0n0l0t0h0':'gcam_decarb'})
dfa = df1[df1.load_zone=='South America_Southern'].copy(deep=True)
dfb = df1[df1.load_zone!='South America_Southern'].copy(deep=True)

dfa['chl_frac']=dfa['period'].map({2020:0.344442,2025:0.335099,2030:0.32497,2035:0.31538,2040:0.306476,2045:0.298366,2050:0.289936})
dfa['pry_frac']=dfa['period'].map({2020:0.088057,2025:0.090023,2030:0.091866,2035:0.093712,2040:0.095528,2045:0.097295,2050:0.099043})
dfa['chl'] = dfa['twh']*dfa['chl_frac']
dfcl = dfa[['scs','load_zone','technology','period','chl']].copy(deep=True).rename(columns={'chl':'twh'})
dfcl['load_zone'] = 'Chile'
dfa['pry'] = dfa['twh']*dfa['pry_frac']
dfpy = dfa[['scs','load_zone','technology','period','pry']].copy(deep=True).rename(columns={'pry':'twh'})
dfpy['load_zone'] = 'Paraguay'

df2 = pd.concat([dfb,dfcl,dfpy])
df2['q'] = df2['technology'].map({'elec_gen':1,'elec_consumption':2,'fe_elec_cons':3})

genco1 = df2[df2.scs=='gcam_ref'].copy(deep=True)
ptgenco1 = genco1.pivot(index=['period','load_zone','q'], columns='technology', values='twh')
ptgenco1['scs'] = 'gcam_ref'
genco2 = df2[df2.scs=='gcam_decarb'].copy(deep=True)
ptgenco2 = genco2.pivot(index=['period','load_zone','q'], columns='technology', values='twh')
ptgenco2['scs'] = 'gcam_decarb'

ptgenco = pd.concat([ptgenco1.reset_index(),ptgenco2.reset_index()])
ptgenco = ptgenco.fillna(0)
ptgenco

technology  period  load_zone  q  elec_consumption    elec_gen  fe_elec_cons  \
0             2020  Argentina  1          0.000000  148.823004      0.000000   
1             2020  Argentina  2        127.828891    0.000000      0.000000   
2             2020  Argentina  3          0.000000    0.000000    127.742361   
3             2020     Brazil  1          0.000000  531.835405      0.000000   
4             2020     Brazil  2        507.860412    0.000000      0.000000   
..             ...        ... ..               ...         ...           ...   
100           2050   Paraguay  2         72.706905    0.000000      0.000000   
101           2050   Paraguay  3          0.000000    0.000000     70.824881   
102           2050    Uruguay  1          0.000000   37.285571      0.000000   
103           2050    Uruguay  2         41.629829    0.000000      0.000000   
104           2050    Uruguay  3          0.000000    0.000000     40.155342   

technology          scs  
0              gcam_ref  
1              gcam_ref  
2              gcam_ref  
3              gcam_ref  
4              gcam_ref  
..                  ...  
100         gcam_decarb  
101         gcam_decarb  
102         gcam_decarb  
103         gcam_decarb  
104         gcam_decarb  

[210 rows x 7 columns]

In [19]:
import plotly.graph_objects as go

edf = ptgenco[ptgenco.period==2050].copy(deep=True).set_index('q')

x = [list(edf.index),
     list(edf.scs)]

fig = go.Figure(go.Bar(x=x,y=list(edf.elec_gen), name='Elec Generation', marker_color=biomass_color))
fig.add_trace(go.Bar(x=x,y=list(edf.elec_consumption), name='Elec Consumption', marker_color=wind_color))
fig.add_trace(go.Bar(x=x,y=list(edf.fe_elec_cons), name='FE Consumption <br>(as Elec)', marker_color=geothermal_color))

#Remove white lines separating elements of bars
fig.update_traces(marker_line_width = 0,
                  selector=dict(type="bar"))

fig.update_layout(barmode="relative",yaxis_title="Electricity generation (TWh)", font = dict(size=13),
                  width=600, height=500, plot_bgcolor='white',#showlegend=False)
                  legend=dict(yanchor="top",y=0.91,xanchor="left", x=1.00), 
                  margin=dict(l=0,r=0,b=0,t=0))

# fig.update_layout(title_text = "B", title_x = 0, title_y=0.99, title_font=dict(size=14))

# Change grid color and axis colors
# fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
fig.update_yaxes(gridcolor='#f5f5f5') # linewidth=2, linecolor='black', 
fig.update_xaxes(tickangle=270)

fig.show()
if save_outputs == 'yes':
    fig.write_image(figpath + 'gen_compare_no_elect_td_bld_{}.png'.format(suffix), scale=4)  #"scale" increases resolution

In [ ]:
import plotly.graph_objects as go

x = [list(df_pt_gen_loss_curt_allscs.index),
     list(df_pt_gen_loss_curt_allscs.scs)]

fig = go.Figure(go.Bar(x=x, y=list(df_pt_gen_loss_curt_allscs.Battery), name='Battery loss', marker_color=battery_color))
fig.add_trace(go.Bar(x=x, y=list(df_pt_gen_loss_curt_allscs.Trans_loss), name='Transmission <br>loss', marker_color=trans_color))
fig.add_trace(go.Bar(x=x, y=list(df_pt_gen_loss_curt_allscs.Coal), name='Coal', marker_color=coal_color))
fig.add_trace(go.Bar(x=x, y=list(df_pt_gen_loss_curt_allscs.Coal_CCS), name='Coal CCS', marker_color=coal_color, marker_pattern_shape="."))
fig.add_trace(go.Bar(x=x, y=list(df_pt_gen_loss_curt_allscs.Gas), name='Gas', marker_color=gas_color))
fig.add_trace(go.Bar(x=x, y=list(df_pt_gen_loss_curt_allscs.Gas_CCS), name='Gas CCS', marker_color=gas_color, marker_pattern_shape="."))
fig.add_trace(go.Bar(x=x, y=list(df_pt_gen_loss_curt_allscs.Nuclear), name='Nuclear', marker_color=nuclear_color))
fig.add_trace(go.Bar(x=x, y=list(df_pt_gen_loss_curt_allscs.Hydro), name='Hydro', marker_color=hydro_color))
fig.add_trace(go.Bar(x=x, y=list(df_pt_gen_loss_curt_allscs.Solar), name='Solar', marker_color=solar_color))
fig.add_trace(go.Bar(x=x, y=list(df_pt_gen_loss_curt_allscs.Wind), name='Wind', marker_color=wind_color))
fig.add_trace(go.Bar(x=x, y=list(df_pt_gen_loss_curt_allscs.Biomass), name='Biomass', marker_color=biomass_color))
fig.add_trace(go.Bar(x=x, y=list(df_pt_gen_loss_curt_allscs.Biomass_CCS), name='Biomass CCS', marker_color=biomass_color, marker_pattern_shape="."))
fig.add_trace(go.Bar(x=x, y=list(df_pt_gen_loss_curt_allscs.Diesel), name='Diesel', marker_color=diesel_color))
fig.add_trace(go.Bar(x=x, y=list(df_pt_gen_loss_curt_allscs.Diesel_CCS), name='Diesel CCS', marker_color=diesel_color, marker_pattern_shape="."))
fig.add_trace(go.Bar(x=x, y=list(df_pt_gen_loss_curt_allscs.Geothermal), name='Geothermal', marker_color=geothermal_color))
fig.add_trace(go.Bar(x=x, y=list(df_pt_gen_loss_curt_allscs.Curtailment), name='Curtailment', marker_color=curtailment_color))

#Remove white lines separating elements of bars
fig.update_traces(marker_line_width = 0,
                  selector=dict(type="bar"))

fig.update_layout(barmode="relative",yaxis_title="Electricity generation (TWh)", font = dict(size=13),
                  width=1000, height=500, plot_bgcolor='white',#showlegend=False)
                  legend=dict(yanchor="top",y=0.91,xanchor="left", x=1.00, traceorder='reversed'), 
                  margin=dict(l=0,r=0,b=0,t=0))

# fig.update_layout(title_text = "B", title_x = 0, title_y=0.99, title_font=dict(size=14))

# Change grid color and axis colors
# fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
fig.update_yaxes(gridcolor='#f5f5f5') # linewidth=2, linecolor='black', 
fig.update_xaxes(tickangle=270)

fig.show()

if save_outputs == 'yes':
    fig.write_image(figpath + 'generation_{}.png'.format(suffix))

# Plot generation anomaly

In [ ]:
### Concat pivot tables of all scenarios
df_pt_gen_loss_curt_allscs = pd.concat([list_pt_gen_loss_curt[i] for i in range(len(scs))])

##list of column names with technologies, exclude last column for scenario name
value_columns = list_pt_gen_loss_curt[0].columns[:-1]

##deduct capacity of Reference scenario from each scenario and assign to a list
list_pt_gen_loss_curt_anom = []

for i in range(len(scs)):
    anom=list_pt_gen_loss_curt[i][value_columns]-df_pt_gen_loss_curt_allscs[df_pt_gen_loss_curt_allscs.scs==ref][value_columns]
    anom['scs'] = scs[i]
    
    list_pt_gen_loss_curt_anom.append(anom)
    
##concat all dfs with delta capacity w.r.t. Reference scenario
df_pt_gen_loss_curt_anom_allscs = pd.concat([list_pt_gen_loss_curt_anom[i] for i in range(len(scs))])

# Shortening Scenario Names (if needed)
for i in range(len(df_pt_gen_loss_curt_anom_allscs)):
    df_pt_gen_loss_curt_anom_allscs.iloc[i,-1] = df_pt_gen_loss_curt_anom_allscs.iloc[i,-1][4:]
for i in range(len(df_pt_gen_loss_curt_allscs)):
    df_pt_gen_loss_curt_allscs.iloc[i,-1] = df_pt_gen_loss_curt_allscs.iloc[i,-1][4:]    

df_pt_gen_loss_curt_anom_allscs.tail()

In [ ]:
##separate existing and new-built gen of Reference scenario
base_sc = df_pt_gen_loss_curt_allscs[(df_pt_gen_loss_curt_allscs.scs == ref_scs_tick)].copy()# &
                                    #(df_pt_gen_loss_curt_allscs.index != 2025) &
                                    #(df_pt_gen_loss_curt_allscs.index != 2035) &
                                    #(df_pt_gen_loss_curt_allscs.index != 2045)].copy()

## separate delta gen of other scenarios, also exclude 2020's data as that is same across all scenarios
alt_sc = df_pt_gen_loss_curt_anom_allscs[(df_pt_gen_loss_curt_anom_allscs.scs != ref_scs_tick) & 
                                      (df_pt_gen_loss_curt_anom_allscs.index != 2020)].copy()# &
                                      #(df_pt_gen_loss_curt_anom_allscs.index != 2025) &
                                      #(df_pt_gen_loss_curt_anom_allscs.index != 2035) &
                                      #(df_pt_gen_loss_curt_anom_allscs.index != 2045)].copy()

## multiply trans and battery loss anomalies with -1 
alt_sc['Trans_loss'] = alt_sc['Trans_loss'].multiply(-1)
alt_sc['Battery'] = alt_sc['Battery'].multiply(-1)


##plot actual gen of Ref. and delta gen of other scenarios

x1 = [list(base_sc.scs),
    list(base_sc.index)]

x2 = [list(alt_sc.index),
    list(alt_sc.scs)]

fig = make_subplots(rows=1, cols=2, column_widths=[0.15, 0.82], shared_yaxes=False, horizontal_spacing=0.15)

##Reference scenario -- Existing gen
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Trans_loss), name='Transmission <br>loss', marker_color=trans_color),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Battery), name='Battery loss', marker_color=battery_color),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Coal), name='Coal', marker_color=coal_color),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Gas), name='Gas', marker_color=gas_color),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Nuclear), name='Nuclear', marker_color=nuclear_color),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Hydro), name='Hydro', marker_color=hydro_color),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Solar), name='Solar', marker_color=solar_color),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Wind), name='Wind', marker_color=wind_color),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Biomass), name='Biomass', marker_color=biomass_color),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Diesel), name='Diesel', marker_color=diesel_color),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Geothermal), name='Geothermal', marker_color=geothermal_color),row=1,col=1)
fig.add_trace(go.Bar(x=x1, y=list(base_sc.Curtailment), name='Curtailment', marker_color=curtailment_color),row=1,col=1)



###Alternative scenarios -- New gen (diff from ref.)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Coal), name='Coal', marker_color=coal_color,showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Coal_CCS), name='Coal CCS', marker_color=coal_color,showlegend=False, marker_pattern_shape="."),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Gas), name='Gas', marker_color=gas_color,showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Gas_CCS), name='Gas CCS', marker_color=gas_color,showlegend=False, marker_pattern_shape="."),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Nuclear), name='Nuclear', marker_color=nuclear_color,showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Hydro), name='Hydro', marker_color=hydro_color,showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Solar), name='Solar', marker_color=solar_color,showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Wind), name='Wind', marker_color=wind_color,showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Biomass), name='Biomass', marker_color=biomass_color,showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Biomass_CCS), name='Biomass CCS', marker_color=biomass_color,showlegend=False, marker_pattern_shape="."),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Diesel), name='Diesel', marker_color=diesel_color,showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Diesel_CCS), name='Diesel CCS', marker_color=diesel_color,showlegend=False, marker_pattern_shape="."),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Geothermal), name='Geothermal', marker_color=geothermal_color,showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Curtailment), name='Curtailment', marker_color=curtailment_color,showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Battery), name='Battery loss', marker_color=battery_color,showlegend=False),row=1,col=2)
fig.add_trace(go.Bar(x=x2, y=list(alt_sc.Trans_loss), name='Transmission <br>loss', marker_color=trans_color,showlegend=False),row=1,col=2)


#Remove white lines separating elements of bars
fig.update_traces(marker_line_width = 0, 
                  selector=dict(type="bar"))

fig.update_layout(barmode="relative", font = dict(size=12),
                  yaxis=dict(title = "Electricity generation (TWh/y)"), 
                  yaxis2=dict(title = "Change in generation (TWh/y) <br> w.r.t. Reference"), 
                  width=910, height=400, plot_bgcolor='white',legend_traceorder="reversed",#showlegend=False)
                  legend=dict(yanchor="top",y=0.9, xanchor="left", x=1.00), margin=dict(l=10,r=10,b=50,t=10))

# fig.update_layout(title_text = "(a)", title_x = 0.5, title_y=0.99, title_font=dict(size=14))

# Change grid color and axis colors
# fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
fig.update_yaxes(gridcolor='#f5f5f5') # linewidth=2, linecolor='black', 

fig.show()

if save_outputs == 'yes':
    fig.write_image(figpath + 'generation_diff_byperiod_{}.png'.format(suffix))

# Gen_share

In [ ]:
df_pt_gen_loss_curt_allscs['fossil_twh'] = df_pt_gen_loss_curt_allscs[['Coal','Gas','Diesel']].sum(axis=1)
df_pt_gen_loss_curt_allscs['fossil_ccs_twh'] = df_pt_gen_loss_curt_allscs[['Coal_CCS','Gas_CCS','Diesel_CCS']].sum(axis=1)

df_pt_gen_loss_curt_allscs['clean_twh'] = df_pt_gen_loss_curt_allscs[['Solar','Wind','Hydro','Geothermal','Nuclear',
                                                                'Coal_CCS','Gas_CCS','Diesel_CCS','Biomass_CCS']].sum(axis=1)

df_pt_gen_loss_curt_allscs['vre_twh'] = df_pt_gen_loss_curt_allscs[['Solar','Wind']].sum(axis=1)

df_pt_gen_loss_curt_allscs['total_twh'] = df_pt_gen_loss_curt_allscs[['Solar','Wind','Hydro','Biomass','Geothermal',
                                                                      'Coal_CCS','Gas_CCS','Diesel_CCS','Biomass_CCS',
                                                                      'Nuclear','Coal','Gas','Diesel']].sum(axis=1)

df_pt_gen_loss_curt_allscs['Hydro_share'] = df_pt_gen_loss_curt_allscs['Hydro'].div(df_pt_gen_loss_curt_allscs['total_twh'])
df_pt_gen_loss_curt_allscs['Gas_share'] = df_pt_gen_loss_curt_allscs['Gas'].div(df_pt_gen_loss_curt_allscs['total_twh'])
df_pt_gen_loss_curt_allscs['Solar_share'] = df_pt_gen_loss_curt_allscs['Solar'].div(df_pt_gen_loss_curt_allscs['total_twh'])
df_pt_gen_loss_curt_allscs['Wind_share'] = df_pt_gen_loss_curt_allscs['Wind'].div(df_pt_gen_loss_curt_allscs['total_twh'])
df_pt_gen_loss_curt_allscs['Clean_share'] = df_pt_gen_loss_curt_allscs['clean_twh'].div(df_pt_gen_loss_curt_allscs['total_twh'])
df_pt_gen_loss_curt_allscs['VRE_share'] = df_pt_gen_loss_curt_allscs['vre_twh'].div(df_pt_gen_loss_curt_allscs['total_twh'])
df_pt_gen_loss_curt_allscs['Fossil_share'] = df_pt_gen_loss_curt_allscs['fossil_twh'].div(df_pt_gen_loss_curt_allscs['total_twh'])
df_pt_gen_loss_curt_allscs['Gas_CCS_share'] = df_pt_gen_loss_curt_allscs['Gas_CCS'].div(df_pt_gen_loss_curt_allscs['total_twh'])
df_pt_gen_loss_curt_allscs['Diesel_CCS_share'] = df_pt_gen_loss_curt_allscs['Diesel_CCS'].div(df_pt_gen_loss_curt_allscs['total_twh'])
df_pt_gen_loss_curt_allscs['Coal_CCS_share'] = df_pt_gen_loss_curt_allscs['Coal_CCS'].div(df_pt_gen_loss_curt_allscs['total_twh'])
df_pt_gen_loss_curt_allscs['Biomass_CCS_share'] = df_pt_gen_loss_curt_allscs['Biomass_CCS'].div(df_pt_gen_loss_curt_allscs['total_twh'])
df_pt_gen_loss_curt_allscs['Fossil_CCS_share'] = df_pt_gen_loss_curt_allscs['fossil_ccs_twh'].div(df_pt_gen_loss_curt_allscs['total_twh'])

df_pt_gen_loss_curt_allscs.round(decimals=2).head(7)

In [ ]:
if save_outputs == 'yes':
    df_pt_gen_loss_curt_allscs.round(decimals=2).to_csv(datapathout + 'genmix_genshare_{}.csv'.format(suffix))

# Capacity and cap_shares

In [ ]:
## export new cap anomalies w.r.t. reference
if save_outputs == 'yes':
    df_pt_ncap1_tech_anom_allscs.round(decimals=2).to_csv(datapathout + 'gencap_anom_{}.csv'.format(suffix))

## estimate cap shares
df_pt_all_cap_tech_allscs['total_gw'] = df_pt_all_cap_tech_allscs[['Biomass', 'Biomass_new', 'Coal', 'Coal_new', 'Diesel',
                                   'Diesel_new', 'Gas', 'Gas_new', 'Geothermal', 'Geothermal_new', 'Hydro',
                                   'Hydro_new', 'Nuclear', 'Nuclear_new', 'Solar', 'Solar_new', 'Wind','Wind_new',
                                    'Coal_CCS','Gas_CCS','Diesel_CCS','Biomass_CCS']].sum(axis=1)

df_pt_all_cap_tech_allscs['total_new_gw'] = df_pt_all_cap_tech_allscs[['Biomass_new', 'Coal_new','Diesel_new', 
                                                                'Gas_new', 'Geothermal_new','Hydro_new', 
                                                                'Nuclear_new', 'Solar_new', 'Wind_new',
                                                                'Coal_CCS','Gas_CCS','Diesel_CCS','Biomass_CCS']].sum(axis=1)

df_pt_all_cap_tech_allscs['all_gas_new'] = df_pt_all_cap_tech_allscs[['Gas_new','Gas_CCS']].sum(axis=1)
df_pt_all_cap_tech_allscs['gas_new_share'] = df_pt_all_cap_tech_allscs['all_gas_new'].div(df_pt_all_cap_tech_allscs['total_new_gw'])

df_pt_all_cap_tech_allscs['hydro_new_share'] = df_pt_all_cap_tech_allscs['Hydro_new'].div(df_pt_all_cap_tech_allscs['total_new_gw'])

df_pt_all_cap_tech_allscs['solar_new_share'] = df_pt_all_cap_tech_allscs['Solar_new'].div(df_pt_all_cap_tech_allscs['total_new_gw'])

df_pt_all_cap_tech_allscs['wind_new_share'] = df_pt_all_cap_tech_allscs['Wind_new'].div(df_pt_all_cap_tech_allscs['total_new_gw'])

df_pt_all_cap_tech_allscs['vre_new_gw'] = df_pt_all_cap_tech_allscs[['Solar_new', 'Wind_new']].sum(axis=1)

df_pt_all_cap_tech_allscs['vre_new_share'] = df_pt_all_cap_tech_allscs['vre_new_gw'].div(df_pt_all_cap_tech_allscs['total_new_gw'])

if save_outputs == 'yes':
    df_pt_all_cap_tech_allscs.round(decimals=2).to_csv(datapathout + 'gencap_newshare_{}.csv'.format(suffix))

df_pt_all_cap_tech_allscs.round(decimals=2).head(7)

## Generation TWh by country

In [ ]:
list_pt_gen_loss_curt = []

for i in range(len(scs)): #len(scs)
    ###Generation-TWh 2020-2050
    alldisp1 = pd.read_csv(respath+''+str(scs[i])+'/results/dispatch_all.csv')    
    alldisp1['twh'] = alldisp1['timepoint_weight'] * alldisp1['power_mw']/(10**6)    
    alldisp1_tec_twh_nstor = alldisp1[(alldisp1.operational_type!='stor')].groupby(['period','technology','load_zone'])['twh'].sum() #& (alldisp1.period!=2045)
    alldisp1_tec_twh_stor = alldisp1[(alldisp1.operational_type=='stor') & (alldisp1.twh<=0)].groupby(['period','technology','load_zone'])['twh'].sum() #& (alldisp1.period!=2045)
    alldisp1_tec_twh_ns = pd.concat([alldisp1_tec_twh_nstor,alldisp1_tec_twh_stor]).reset_index()
    
    ###Trans_Loss 2020-2050
    trans = pd.read_csv(respath+''+str(scs[i])+'/results/transmission_operations.csv')
    trans['twh'] = (trans['transmission_losses_lz_from']+trans['transmission_losses_lz_to'])*trans['timepoint_weight']/(10**6)
    df_trans_loss = pd.DataFrame([trans.groupby(['period','lz_from'])['twh'].sum()*(-1)]).transpose() #[(trans.period!=2045)]
    df_trans_loss['technology']='Trans_loss'
    df_trans_loss = df_trans_loss.reset_index()

    ###Curtailment-VRE 2020-2050
    disp_re = pd.read_csv(respath+''+str(scs[i])+'/results/dispatch_variable.csv')
    disp_re['twh'] = disp_re['total_curtailment_mw']*disp_re['timepoint_weight']/(10**6)
    curt_re = disp_re.groupby(['period','load_zone'])['twh'].sum() #[(disp_re.period!=2045)]
    curt_re = curt_re.reset_index()
    curt_re['technology']='curt_vre'

    ###Curtailment-Hydro 2020-2050
    disp_hydro = pd.read_csv(respath+''+str(scs[i])+'/results/dispatch_gen_hydro.csv')
    disp_hydro['twh'] = disp_hydro['scheduled_curtailment_mw']*disp_hydro['timepoint_weight']/(10**6)
    curt_hydro = disp_hydro.groupby(['period','load_zone'])['twh'].sum() #[(disp_hydro.period!=2045)]
    curt_hydro = curt_hydro.reset_index()
    curt_hydro['technology']='curt_hydro'

    ###Concat Gen_TWh, Trans_loss, and Curtailments
    df_gen_loss_curt = pd.concat([alldisp1_tec_twh_ns, df_trans_loss, curt_re, curt_hydro],axis=0)
    df_gen_loss_curt.set_index('period','load_zone').sort_index()

    ###rename tech for plotting
    df_gen_loss_curt.loc[df_gen_loss_curt['technology'] == 'Coal_Sub','technology'] = 'Coal'
    df_gen_loss_curt.loc[df_gen_loss_curt['technology'] == 'Coal_Sup','technology'] = 'Coal'
    df_gen_loss_curt.loc[df_gen_loss_curt['technology'] == 'Coal_IGCC','technology'] = 'Coal'
    df_gen_loss_curt.loc[df_gen_loss_curt['technology'] == 'Gas_CCGT','technology'] = 'Gas'
    df_gen_loss_curt.loc[df_gen_loss_curt['technology'] == 'Gas_OCGT','technology'] = 'Gas'
    df_gen_loss_curt.loc[df_gen_loss_curt['technology'] == 'Gas_ICE','technology'] = 'Gas'
    df_gen_loss_curt.loc[df_gen_loss_curt['technology'] == 'Diesel','technology'] = 'Diesel'
    df_gen_loss_curt.loc[df_gen_loss_curt['technology'] == 'Oil','technology'] = 'Diesel'
    df_gen_loss_curt.loc[df_gen_loss_curt['technology'] == 'Geothermal','technology'] = 'Geothermal'
    df_gen_loss_curt.loc[df_gen_loss_curt['technology'] == 'Biomass','technology'] = 'Biomass'
    df_gen_loss_curt.loc[df_gen_loss_curt['technology'] == 'SolarPV','technology'] = 'Solar'
    df_gen_loss_curt.loc[df_gen_loss_curt['technology'] == 'curt_vre','technology'] = 'Curtailment'
    df_gen_loss_curt.loc[df_gen_loss_curt['technology'] == 'curt_hydro','technology'] = 'Curtailment'

    ##sum of twh by renamed tech and period
    df_gen_loss_curt_reclass = df_gen_loss_curt.groupby(['period','technology','load_zone'])['twh'].sum().reset_index()

    ###Creat pivot table
    pt_gen_loss_curt = df_gen_loss_curt_reclass.pivot(index=['period','load_zone'], columns='technology', values='twh')

    ###assign scenario name in a column
    pt_gen_loss_curt['scs'] = scs[i]

    ###Append to the pivot table to list
    list_pt_gen_loss_curt.append(pt_gen_loss_curt.reset_index())

len(list_pt_gen_loss_curt)

In [ ]:
### Concat pivot tables of all scenarios
df_pt_gen_loss_curt_allscs = pd.concat([list_pt_gen_loss_curt[i] for i in range(len(scs))])
df_pt_gen_loss_curt_allscs = df_pt_gen_loss_curt_allscs.set_index('period')

# Shortening Scenario Names (if needed)
for i in range(len(df_pt_all_cap_tech_allscs)):
    df_pt_gen_loss_curt_allscs.iloc[i,-1] = df_pt_gen_loss_curt_allscs.iloc[i,-1][4:]
ref_scs_tick = ref[4:]

df_pt_gen_loss_curt_allscs = df_pt_gen_loss_curt_allscs[(df_pt_gen_loss_curt_allscs.index==2020) |
                                                        (df_pt_gen_loss_curt_allscs.index==2025) |
                                                        (df_pt_gen_loss_curt_allscs.index==2030) |
                                                        (df_pt_gen_loss_curt_allscs.index==2035) |
                                                        (df_pt_gen_loss_curt_allscs.index==2040) |
                                                        (df_pt_gen_loss_curt_allscs.index==2045) |
                                                        (df_pt_gen_loss_curt_allscs.index==2050)].copy()

# Gen Share by country

In [ ]:
df_pt_gen_loss_curt_allscs['fossil_twh'] = df_pt_gen_loss_curt_allscs[['Coal','Gas','Diesel']].sum(axis=1)
df_pt_gen_loss_curt_allscs['fossil_ccs_twh'] = df_pt_gen_loss_curt_allscs[['Coal_CCS','Gas_CCS','Diesel_CCS']].sum(axis=1)

df_pt_gen_loss_curt_allscs['clean_twh'] = df_pt_gen_loss_curt_allscs[['Solar','Wind','Hydro','Geothermal','Nuclear',
                                                                'Coal_CCS','Gas_CCS','Diesel_CCS','Biomass_CCS']].sum(axis=1)

df_pt_gen_loss_curt_allscs['vre_twh'] = df_pt_gen_loss_curt_allscs[['Solar','Wind']].sum(axis=1)

df_pt_gen_loss_curt_allscs['total_twh'] = df_pt_gen_loss_curt_allscs[['Solar','Wind','Hydro','Biomass','Geothermal',
                                                                      'Coal_CCS','Gas_CCS','Diesel_CCS','Biomass_CCS',
                                                                      'Nuclear','Coal','Gas','Diesel']].sum(axis=1)

df_pt_gen_loss_curt_allscs['Hydro_share'] = df_pt_gen_loss_curt_allscs['Hydro'].div(df_pt_gen_loss_curt_allscs['total_twh'])
df_pt_gen_loss_curt_allscs['Gas_share'] = df_pt_gen_loss_curt_allscs['Gas'].div(df_pt_gen_loss_curt_allscs['total_twh'])
df_pt_gen_loss_curt_allscs['Solar_share'] = df_pt_gen_loss_curt_allscs['Solar'].div(df_pt_gen_loss_curt_allscs['total_twh'])
df_pt_gen_loss_curt_allscs['Wind_share'] = df_pt_gen_loss_curt_allscs['Wind'].div(df_pt_gen_loss_curt_allscs['total_twh'])
df_pt_gen_loss_curt_allscs['Clean_share'] = df_pt_gen_loss_curt_allscs['clean_twh'].div(df_pt_gen_loss_curt_allscs['total_twh'])
df_pt_gen_loss_curt_allscs['VRE_share'] = df_pt_gen_loss_curt_allscs['vre_twh'].div(df_pt_gen_loss_curt_allscs['total_twh'])
df_pt_gen_loss_curt_allscs['Fossil_share'] = df_pt_gen_loss_curt_allscs['fossil_twh'].div(df_pt_gen_loss_curt_allscs['total_twh'])
df_pt_gen_loss_curt_allscs['Gas_CCS_share'] = df_pt_gen_loss_curt_allscs['Gas_CCS'].div(df_pt_gen_loss_curt_allscs['total_twh'])
df_pt_gen_loss_curt_allscs['Diesel_CCS_share'] = df_pt_gen_loss_curt_allscs['Diesel_CCS'].div(df_pt_gen_loss_curt_allscs['total_twh'])
df_pt_gen_loss_curt_allscs['Coal_CCS_share'] = df_pt_gen_loss_curt_allscs['Coal_CCS'].div(df_pt_gen_loss_curt_allscs['total_twh'])
df_pt_gen_loss_curt_allscs['Biomass_CCS_share'] = df_pt_gen_loss_curt_allscs['Biomass_CCS'].div(df_pt_gen_loss_curt_allscs['total_twh'])
df_pt_gen_loss_curt_allscs['Fossil_CCS_share'] = df_pt_gen_loss_curt_allscs['fossil_ccs_twh'].div(df_pt_gen_loss_curt_allscs['total_twh'])

df_pt_gen_loss_curt_allscs.round(decimals=3)

In [ ]:
if save_outputs == 'yes':
    df_pt_gen_loss_curt_allscs.round(decimals=4).to_csv(datapathout + 'genmix_genshare_bycountry_{}.csv'.format(suffix))